#Start

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Import

In [ ]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time
import re

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

# Headers

In [ ]:
headers_0 = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;x64; rv:66.0) Gecko/20100101 Firefox/66.0",
           "Accept-Encoding":"gzip, deflate",
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "DNT":"1",
           "Connection":"close",
           "Upgrade-Insecure-Requests":"1"}

headers_1 = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5)AppleWebKit/605.1.15 (KHTML, like Gecko)Version/12.1.1 Safari/605.1.15",
           "Accept-Encoding":"gzip, deflate",
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "DNT":"1",
           "Connection":"close",
           "Upgrade-Insecure-Requests":"1"}

headers_2 = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
           "Accept-Encoding":"gzip, deflate",
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "DNT":"1",
           "Connection":"close",
           "Upgrade-Insecure-Requests":"1"}

headers_tuple = (headers_1, headers_1, headers_2)

In [ ]:
# # def headers_walk(headers_tuple=headers_tuple, spot):
# header_count = 0
# headers_tuple[header_count]
# header_count += 1
# if header_count > 3:
#     header_count = 0

# Extract soup

In [ ]:
def extract_soup(url, header=0, preview=True):
    headers = headers_tuple[header]
    response = requests.get(url, headers=headers)
    status = response.status_code

    soup = BeautifulSoup(response.text, 'lxml')

    if preview==True:
        print(soup.prettify())

    return soup, status

In [ ]:
    # test_url = 'https://www.amazon.com.mx/gp/bestsellers/musical-instruments/ref=zg_bs_nav_0'
    # test_soup, test_status = extract_soup(test_url, False)
    # print(test_status)
    # print(type(test_status))

#Top amazon boxes

In [ ]:
def top_amazon_boxes(soup):
    boxes = soup.find_all('div', attrs={'class':"a-section a-spacing-none aok-relative"})

    return boxes

#Centered String

In [ ]:
def centered_string(string):
    rest = 21 - len(string)
    n_spaces = int(rest/2)

    string_spaces = ' ' * n_spaces
    
    centered_string = string_spaces + string + string_spaces

    if len(centered_string)<21:
        centered_string = ' ' + centered_string

    return centered_string

#Update CSV

In [ ]:
def update_csv(file, rows, sep=','):
    with open(file, "a", encoding="utf-8") as csv_file:
        csv_file = csv.writer(csv_file, delimiter=sep)
        for row in rows:
            csv_file.writerow(row)

# Product ID

In [ ]:
def product_id(complete_product_link):
    regex_a = '(?<=dp\/)(.+)(?=\/ref)'
    regex_b = '(?<=dp\/)(.+)(?=\?)'

    product_link_a = re.search(regex_a, complete_product_link)
    if product_link_a is None:

        product_link_b = re.search(regex_b, complete_product_link)
        if product_link_b is None:
            product_id = complete_product_link
        else:
            product_id = product_link_b.group()
    else:
        product_id = product_link_a.group()
    
    return product_id

# Image ID

In [ ]:
def image_id(complete_image_link):
    regex_a = '(?<=I\/)(.*)(?=\.jpg)'
    image_link_a = re.search(regex_a, complete_image_link)

    if not image_link_a:
        image_id = complete_image_link
    else:
        image_id = image_link_a.group()
    
    return image_id

#Scrap boxes

In [ ]:
def scrap_boxes(boxes, domain, dict_to_rows=False):
    
    ranks = [None]*50
    product_names = [None]*50
    image_urls = [None]*50
    product_links = [None]*50
    star_ratings = [None]*50
    reviews = [None]*50
    authors_companies = [None]*50
    editions_consoles = [None]*50
    min_prices = [None]*50
    max_prices = [None]*50
    time_log = [None]*50

    amz_mx_url = f'https://www.amazon.com.{domain}'
    
    n_box = 0
    for box in boxes:
        MX__COL_tz = 'America/Mexico_City'
        timezone_MXCOL = timezone(MX__COL_tz) 
        datetime_box = datetime.now(timezone_MXCOL)
        time_log[n_box] = datetime_box.strftime('%Y-%m-%d %H:%M')

        rank_box = box.find_all('span', attrs={'class':'zg-badge-text'})
        products_and_image_box = box.find_all('div', attrs={'class' : 'a-section a-spacing-small'})
        product_links_box = box.find_all('a', attrs={'class' : 'a-link-normal'})
        star_ratings_box = box.find_all('span', attrs={'class' : 'a-icon-alt'})
        reviews_box = box.find_all('a', attrs={'class' : 'a-size-small a-link-normal'})
        authors_company_box = box.find_all('span', attrs={'class' : 'a-size-small a-color-base'})
        editions_console_box = box.find_all('span', attrs={'class' : 'a-size-small a-color-secondary'})
        prices_box = box.find_all('span', attrs={'class' : "p13n-sc-price"})
        
        
        ranks[n_box] = int(rank_box[0].get_text()[1:])

        #In case the element was removed (yes, it happens)
        try:
            product_names[n_box] = products_and_image_box[0].img.get('alt')
        except: pass

        try:
            complete_product_link = product_links_box[0].get('href')
            product_links[n_box] = product_id(complete_product_link)
        except: pass

        try:
            complete_image_id = products_and_image_box[0].img.get('src')
            image_urls[n_box] = image_id(complete_image_id)
        except: pass

        try:
            if domain == 'mx':
                star_ratings[n_box] = float(star_ratings_box[0].get_text()[:3])
                reviews[n_box] = int(reviews_box[0].get_text().replace(',',''))

            elif domain == 'br':
                star_ratings[n_box] = float(star_ratings_box[0].get_text()[:3].replace(',','.'))
                reviews[n_box] = int(reviews_box[0].get_text().replace('.',''))
        except: pass
        #Individual cases
        try:
            authors_companies[n_box] = authors_company_box[0].get_text()
        except:pass
        
        try:
            editions_consoles[n_box] = editions_console_box[0].get_text()
        except:pass

        #Courrencies
        if domain == 'mx':
            coin_symbol = 1
        elif domain == 'br':
            coin_symbol = 2

        try:
            if domain == 'mx':
                min_prices[n_box] = float(prices_box[0].get_text()[coin_symbol:].replace(',',''))
            elif domain == 'br':
                min_prices[n_box] = float(prices_box[0].get_text()[coin_symbol:].replace('.','').replace(',','.'))
        except: pass

        try:    
            if domain == 'mx':
                max_prices[n_box] = float(prices_box[1].get_text()[coin_symbol:].replace(',',''))
            elif domain == 'br':
                max_prices[n_box] = float(prices_box[1].get_text()[coin_symbol:].replace('.','').replace(',','.'))
        except: pass

        n_box = n_box + 1 

    # Dictionary
    boxes_dict = {
    'time' : time_log,
    "Rank" : ranks,
    "Product Names": product_names,
    "Product ID": product_links,
    "Image ID": image_urls,
    "Stars": star_ratings,
    "Reviews": reviews,
    "Authors/Company": authors_companies,
    "Edition/Console": editions_consoles,
    "Price_std_or_min" : min_prices,
    "Max_prices" : max_prices
    }

    if dict_to_rows == True:
        dict_rows = [None]*50
        for n in range(len(dict_rows)):
            dict_rows[n] = [
                boxes_dict["time"][n],
                boxes_dict["Rank"][n],
                boxes_dict["Product Names"][n],
                boxes_dict["Product ID"][n],
                boxes_dict["Image ID"][n],
                boxes_dict["Stars"][n],
                boxes_dict["Reviews"][n],
                boxes_dict["Authors/Company"][n],
                boxes_dict["Edition/Console"][n],
                boxes_dict["Price_std_or_min"][n],
                boxes_dict["Max_prices"][n]]

        return boxes_dict, dict_rows
    else:
        return boxes_dict

##mx dictionary

In [ ]:
mx_dict = {
    'top_food_and_drinks':'grocery',
    'top_automotive':'automotive',
    'top_baby':'baby',
    'top_sports':'sports',
    'top_amazon_devices':'amazon-devices',
    'top_electronics':'electronics',
    'top_tools':'tools',
    'top_kitchen':'kitchen',
    'top_industrial_emp_cience':'industrial',
    'top_musical_instruments':'musical-instruments',
    'top_toys':'toys',
    'top_books':'books',
    'top_music':'music',
    'top_officeproduct':'officeproduct',
    'top_movies':'dvd',
    'top_handmade':'handmade',
    'top_pet_supplies':'pet-supplies',
    'top_fashion':'shoes',
    'top_health':'hpc',
    'top_software':'software',
    'top_kindle':'digital-text',
    'top_videogames':'videogames'
    }

##br ditionary

In [ ]:
br_dict = {
    'top_amazon_devices':'amazon-devices',
    'top_automotivo': 'automotive',
    'top_food_and_drinks':'grocery',
    'top_baby':'baby-products',
    'top_sports':'sports',
    'top_electronics':'electronics',
    'top_tools':'hi',
    'top_kitchen':'kitchen',
    'top_toys':'toys',
    'top_books':'books',
    'top_music':'music',
    'top_officeproduct':'office',
    'top_movies':'dvd',
    'top_pet_supplies':'pet-products',
    'top_fashion':'fashion',
    'top_health':'hpc',
    'top_software':'computers',
    'top_kindle':'digital-text',
    'top_videogames':'videogames',
    'top_apps':'mobile-apps',
    'top_beauty':'beauty',
    'top_home':'home',
    'top_home_appliances':'appliances',
    'top_pool_garden':'lawn-and-garden',
    'top_furniture':'furniture'
}

##domain dictionary

In [ ]:
domain_dict = {'Mexico' : 'mx',
               'Brazil' : 'br'}

##domain path

In [ ]:
domain_path = '4SS/4SS_db/testing/{}'

# Test Amazon Scrap

In [ ]:
def test_scrap_amazon(country):
    if country == 'mx':
        categories_dict = mx_dict
        n_spaces = 45

    elif country == 'br':
        categories_dict = br_dict
        n_spaces = 47

    log_status = [None]*n_spaces
    log_date = [None]*n_spaces
    log_domain = [None]*n_spaces
    log_category = [None]*n_spaces
    n_header = 0

    #Category that is scraping
    key = 'top_music'
    category = categories_dict[key]
    
    #The main scrap
    url = f'https://www.amazon.com.{country}/gp/bestsellers/{category}/ref=zg_bs_nav_0'
    soup, status = extract_soup(url,n_header, preview=False)

    if status == 503:
        while status == 503:
            time.sleep(1)
            soup, status = extract_soup(url, preview=False)
            log_status[n_log] = status
            log_date[n_log] = datetime.now()

    if status ==200:
        boxes = top_amazon_boxes(soup)
        amz_key_top_boxes, amz_rows = scrap_boxes(boxes, country, dict_to_rows=True)

        for key in amz_key_top_boxes:
            print(key, end=' | ')

        print('\n')

        for row in amz_rows:
            for idx in range(len(row)):        
                print(row[idx], end=' | ')
            print('\n')

#scrap amazon

In [ ]:
def scrap_amazon():
    MX__COL_tz = 'America/Mexico_City'
    timezone_MXCOL = timezone(MX__COL_tz) 

    Start_datetime = datetime.now(timezone_MXCOL)
    Start_datetime = Start_datetime.strftime('%Y-%m-%d %H_%M')
    centered_date = centered_string(Start_datetime)

    n_log = 0
    country_count = 0

    print(f'      Log Date    | Country |        category       | header | status | Loaded |')

    for country_name in domain_dict:
        country = domain_dict[country_name]

        if country == 'mx':
            categories_dict = mx_dict
            n_spaces = 45

        elif country == 'br':
            categories_dict = br_dict
            n_spaces = 47

        log_status = [None]*n_spaces
        log_date = [None]*n_spaces
        log_domain = [None]*n_spaces
        log_category = [None]*n_spaces
        n_header = 0

        country_count = country_count + 1

        if country_count > 1:
            print(' ---------------- | ------- | --------------------- | ------ | ------ | ------ |')


        for key in categories_dict:
            #Date-time
            log_datetime = datetime.now(timezone_MXCOL)
            log_date[n_log] = log_datetime.strftime('%Y-%m-%d %H:%M')
            print(f' {log_date[n_log]}', end=' | ')
            
            #Country where you are at
            log_domain[n_log] = country
            print(f'   {log_domain[n_log]}  ', end=' | ')

            #Category that is scraping
            category = categories_dict[key]
            log_category[n_log] = category
            centered_category = centered_string(log_category[n_log])
            print(centered_category, end=' | ')
            
            #The main scrap
            url = f'https://www.amazon.com.{country}/gp/bestsellers/{category}/ref=zg_bs_nav_0'

            
            print(f'  {n_header}   ', end=' | ')

            soup, status = extract_soup(url,n_header, preview=False)
            n_header += 1
            if n_header > 2:
                n_header = 0

            if status == 503:
                while status == 503:
                    time.sleep(1)
                    soup, status = extract_soup(url, preview=False)
                    log_status[n_log] = status
                    log_date[n_log] = datetime.now()

            if status ==200:
                log_status[n_log] = status
                print(f'  {log_status[n_log]} ', end=' | ')
                
                boxes = top_amazon_boxes(soup)
                amz_key_top_boxes, amz_rows = scrap_boxes(boxes, country, dict_to_rows=True)
                add_df = pd.DataFrame.from_dict(amz_key_top_boxes)

                #Updating Parquet files
                bdb = f'/{country}-{category}.parquet'
                dir_parquet_maindb = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{country}/parquet/{bdb}'
                main_df = pd.read_parquet(dir_parquet_maindb)
                main_df = pd.concat([main_df, add_df])
                #case of NaN values
                main_df['Rank'] = pd.to_numeric(main_df['Rank'], downcast='float')

                parquet_file = f'/{country}-{category}.parquet'
                dir_parquet_maindb = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{country}/parquet/{parquet_file}'
                #Updating  file
                main_df.to_parquet(dir_parquet_maindb)

                excel_file = f'/{country}-{category}.xlsx'
                dir_excel_maindb = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{country}/excel/{excel_file}'
                #For the testing
                main_df.to_excel(dir_excel_maindb, index=False)

                #Updating csv files
                csv_file = f'/{country}-{category}.csv'
                dir_csv_maindb = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{country}/csv/{csv_file}'

                print('  Yes  |')

            else:
                log_status[n_log] == status
                print(f'  {log_status[n_log]} ', end=' | ')
                
                log_date[n_log] = datetime.now()
                print(' +.No.+ |')

            n_log = n_log + 1

    log_dict = {
        'log_date' : log_date,
        'category' : log_category,
        'country' : log_domain,
        'status' : log_status
    }


    dict_rows= [
        log_date,
        log_category,
        log_domain,
        log_status]

    log_file = f'/master_db_logs.csv'
    dir_log_testing = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{log_file}'

    try:
        update_csv(dir_log_testing, log_rows, sep='|')
    except:
        log_df = pd.DataFrame.from_dict(log_dict)
        log_df.to_csv(dir_log_testing, index = False)



    log_print = """
                                ----------------------
                                |   Log file loaded   |
                                ----------------------

    """
    print(log_print)

#Scrap Amazon n times

In [ ]:
def scrap_amazon_times(n_times, minutes):
    counter = 0
    for t in range(n_times):
        for min in range(minutes):
            print(min, end='')
            line_jump = 0
            for wait in range(3):
                time.sleep(15)
                print('.', end='')
                if (min > 0) and (min % 20 == 0) and (line_jump == 0):
                    line_jump = 1
                    print('\n')
        
        print(f'\nScrap No: {counter + 1}\n')
        scrap_amazon()
        counter = counter + 1

#Wait this time

In [ ]:
#time.sleep(60*30)
#scrap_amazon()

#Scrap n times in t time or just one time

In [ ]:
test = False
if test == True:
    test_scrap_amazon(country='mx')
else:
    scrap_amazon()

In [ ]:
case = 'steps'
#case = 'normal'

if case == 'steps':
    scrap_amazon_times(n_times= 11, minutes= 60)
else:
    scrap_amazon()

0...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20.

..21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40.

..41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...58...59...
Scrap No: 1

      Log Date    | Country |        category       | header | status | Loaded |
 2020-10-01 21:40 |    mx   |        grocery        |   0    |   200  |   Yes  |
 2020-10-01 21:40 |    mx   |       automotive      |   1    |   200  |   Yes  |
 2020-10-01 21:40 |    mx   |          baby         |   2    |   200  |   Yes  |
 2020-10-01 21:40 |    mx   |         sports        |   0    |   200  |   Yes  |
 2020-10-01 21:41 |    mx   |     amazon-devices    |   1    |   200  |   Yes  |
 2020-10-01 21:41 |    mx   |      electronics      |   2    |   200  |   Yes  |
 2020-10-01 21:41 |    mx   |         tools         |   0    |   200  |   Yes  |
 2020-10-01 21:41 |    mx   |        kitchen